# MINIMIZE_SCALAR

## Overview
The `MINIMIZE_SCALAR` function finds the minimum value of a scalar mathematical function $f(x)$, where $x$ is a real number (i.e., a function of a single variable only). This is useful for business users in Excel who need to optimize costs, maximize efficiency, or find the best value for a given scenario. The function leverages [`scipy.optimize.minimize_scalar`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize_scalar.html) and allows users to specify the function as a string, with optional bounds and method. The minimization problem can be stated as:

```math
\min_x f(x)
```

where $f(x)$ is the user-provided function expression. If bounds are provided, the minimization is performed over the interval $[a, b]$.

This example function is provided as-is without any representation of accuracy.

## Usage
To use the function in Excel:
```excel
=MINIMIZE_SCALAR(func_expr, [bounds], [method])
```
- `func_expr` (string, required): The function to minimize, as a string (e.g., `"x**2 + 3*x + 2"`). Example: `"x**2 + 3*x + 2"`
- `bounds` (2D list, optional): 2D list `[[min, max]]` for bounded minimization. Example: `[[0, 10]]`
- `method` (string, optional): Optimization method: `"brent"`, `"bounded"`, or `"golden"`. Example: `"bounded"`

The function returns a 2D list `[[x, fun]]` where `x` is the location of the minimum and `fun` is the minimum value, or a string error message if input is invalid or an error occurs.

## Examples

**Example 1: Minimize a Quadratic Cost Function**

This example minimizes $x^2 + 3x + 2$ with no bounds.

In Excel:
```excel
=MINIMIZE_SCALAR("x**2 + 3*x + 2")
```
Expected output:

| x    | fun   |
|------|-------|
| -1.5 | -0.25 |

**Example 2: Minimize a Function with Bounds**

This example minimizes $(x-5)^2 + 10$ over $[0, 10]$ using the `bounded` method.

In Excel:
```excel
=MINIMIZE_SCALAR("(x-5)**2 + 10", {0,10}, "bounded")
```
Expected output:

| x   | fun  |
|-----|------|
| 5.0 | 10.0 |

In [ ]:
from scipy.optimize import minimize_scalar as scipy_minimize_scalar
import math

def minimize_scalar(func_expr, bounds=None, method=None):
    """
    Minimizes a scalar function using scipy.optimize.minimize_scalar.

    Args:
        func_expr (str): The function to minimize, as a string (e.g., 'x**2 + 3*x + 2').
        bounds (list, optional): 2D list [[min, max]] for bounded minimization.
        method (str, optional): Optimization method: 'brent', 'bounded', or 'golden'.

    Returns:
        list: [[x, fun]] where x is the location of minimum and fun is the minimum value, or a string error message if input is invalid or an error occurs.

    This example function is provided as-is without any representation of accuracy.
    """
    # Validate func_expr
    if not isinstance(func_expr, str):
        return "func_expr must be a string."
    if 'x' not in func_expr:
        return "Function expression must contain the variable 'x'."
    # Validate method
    if method is not None and not isinstance(method, str):
        return "method must be a string or None."
    # Validate bounds
    if bounds is not None:
        if not (isinstance(bounds, list) and len(bounds) == 1 and isinstance(bounds[0], list) and len(bounds[0]) == 2):
            return "bounds must be a 2D list [[min, max]] or None."
    # Method/bounds logic
    if method == 'bounded':
        if bounds is None:
            return "Method 'bounded' requires bounds to be specified."
    elif method in ['brent', 'golden']:
        if bounds is not None:
            return f"Method '{method}' cannot be used with bounds. Use 'bounded' instead."
    # Define function
    def func(x):
        try:
            return eval(func_expr, {"x": x, "math": math})
        except Exception:
            return float('nan')
    # Prepare kwargs
    kwargs = {}
    if bounds is not None:
        min_val, max_val = bounds[0][0], bounds[0][1]
        kwargs['bounds'] = (min_val, max_val)
    if method is not None:
        kwargs['method'] = method
    # Run minimization
    try:
        result = scipy_minimize_scalar(func, **kwargs)
        if not hasattr(result, 'success') or not result.success or math.isnan(result.x) or math.isnan(result.fun):
            return f"Error during minimization: {getattr(result, 'message', 'Unknown error')}"
        return [[float(result.x), float(result.fun)]]
    except Exception as e:
        return f"Error during minimization: {str(e)}"

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()
import pytest

demo_cases = [
    ["x**2 + 3*x + 2", None, "brent", [[-1.4999999999999716, -0.25]]],
    ["(x-5)**2 + 10", [[0, 10]], "bounded", [[5.0, 10.0]]],
    ["x**2 + 3*x + 2", [[-10, 10]], "bounded", [[-1.5, -0.25]]]
]

def approx_equal(a, b, rel=0.05):
    # Scalar float only
    if isinstance(a, float) and isinstance(b, float):
        return a == pytest.approx(b, rel=rel)
    # 2D list of floats only
    if (
        isinstance(a, list) and isinstance(b, list)
        and all(isinstance(x, list) for x in a)
        and all(isinstance(y, list) for y in b)
    ):
        return all(
            all(isinstance(x, float) and isinstance(y, float) and x == pytest.approx(y, rel=rel) for x, y in zip(row_a, row_b))
            for row_a, row_b in zip(a, b)
        )
    return False

@pytest.mark.parametrize("func_expr, bounds, method, expected", demo_cases)
def test_demo_cases(func_expr, bounds, method, expected):
    result = minimize_scalar(func_expr, bounds, method)
    print(f"test_demo_cases output for {func_expr}: {result}")
    assert approx_equal(result, expected, rel=0.05), f"Output {result} not within 5% of expected {expected}"

def test_invalid_expression():
    result = minimize_scalar("x***2 + 3x + 2", None, None)
    assert isinstance(result, str) and ("error" in result.lower() or "must be" in result.lower())

def test_missing_x():
    result = minimize_scalar("5 + 7", None, None)
    assert isinstance(result, str) and ("function expression must contain" in result.lower())

ipytest.run('-s')

In [ ]:
# Interactive Demo

import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
import io
import base64

def gradio_minimize_scalar(func_expr, bounds, method):
    print(f"gradio_minimize_scalar args: func_expr={func_expr}, bounds={bounds}, method={method}")
    # Convert bounds to float if possible for both plotting and minimize_scalar
    bounds_float = None
    if bounds and isinstance(bounds, list) and len(bounds) == 1 and len(bounds[0]) == 2:
        try:
            min_b = float(bounds[0][0])
            max_b = float(bounds[0][1])
            bounds_float = [[min_b, max_b]]
        except Exception:
            bounds_float = None
    else:
        bounds_float = None

    result = minimize_scalar(func_expr, bounds_float, method)
    # Prepare plot
    x_vals = np.linspace(-10, 10, 400)
    if bounds_float:
        x_vals = np.linspace(bounds_float[0][0], bounds_float[0][1], 400)
    y_vals = []
    for x in x_vals:
        try:
            y = eval(func_expr, {"x": x, "math": __import__('math')})
        except Exception:
            y = np.nan
        y_vals.append(y)
    plt.figure(figsize=(6, 4))
    plt.plot(x_vals, y_vals, label="f(x)")
    if isinstance(result, list) and len(result) > 0 and isinstance(result[0], list) and len(result[0]) == 2:
        min_x, min_y = result[0]
        plt.scatter([min_x], [min_y], color="red", label="Minimum")
        plt.annotate(f"min=({min_x:.3g}, {min_y:.3g})", (min_x, min_y), textcoords="offset points", xytext=(0,10), ha='center', color='red')
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.title("Function Minimization")
    plt.legend()
    plt.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    plt.close()
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode("utf-8")
    img_html = f'<img src="data:image/png;base64,{img_base64}" style="max-width:100%;height:auto;" />'
    # Prepare result for DataFrame and HTML
    if isinstance(result, str):
        df_result = None
        html_result = f'<div style="color:red;font-weight:bold;">{result}</div>'
    else:
        df_result = result
        html_result = img_html
    return df_result, html_result

demo = gr.Interface(
    fn=gradio_minimize_scalar,
    inputs=[
        gr.Textbox(label="Function Expression", value=demo_cases[0][0]),
        gr.DataFrame(headers=["min", "max"], label="Bounds (optional)", row_count=1, col_count=2, type="array", value=demo_cases[0][1], visible=False),
        gr.Dropdown(
            choices=["brent", "bounded", "golden", ""],
            label="Method (optional)",
            value=demo_cases[0][2] if demo_cases[0][2] else "brent",
            allow_custom_value=False,
            visible=False,
        ),
    ],
    outputs=[
        gr.DataFrame(headers=["x", "f(x)"], label="Result (x, f(x))"),
        gr.HTML(label="Plot"),
    ],
    examples=demo_cases,
    flagging_mode="never",
    fill_width=True,
)
demo.launch()